In [1]:
import rospy
import actionlib
import actionlib.msg
import assignment_2_2023.msg
from std_srvs.srv import *
import sys
import select
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Point, Pose, Twist
from assignment_2_2023.msg import Vel
from assignment_2_2023.msg import PlanningAction, PlanningGoal, PlanningResult, PlanningActionResult
from actionlib_msgs.msg import GoalStatus
from sensor_msgs.msg import LaserScan
import math
import ipywidgets as widgets
from IPython.display import display 
from ipywidgets import Layout, HBox
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import interactive, fixed
import tf
from nav_msgs.msg import Odometry
from tf.transformations import quaternion_matrix
import numpy as np

In [2]:
# Initialize global variables
past_goals = []
last_goal = None  # Keep track of the last goal

# keep track of reached/unreached goals
reached_goals = 0
unreached_goals = 0
counter = 0

global goal_reached_flag
goal_reached_flag = False


# Define state descriptions
state_desc_ = {
    0: 'Go to point',
    1: 'Wall following',
    2: 'Done',
    6: 'Goal cancelled'
}


font_family = {'family': 'arial', 'color':  'navy', 'weight': 'bold', 'size': 20}
font_family_smaller = {'family': 'arial', 'color':  'navy', 'weight': 'bold', 'size': 15}



In [3]:
# Check if ROS node has already been initialized
if not rospy.core.is_initialized():
    rospy.init_node('set_target_client')

In [4]:
# Declare position_display in the global scope
position_display = widgets.HTML(value="<b>POSITION</b>: <b>x</b>: 0, <b>y</b>: 0<br><b>VELOCITY</b>: <b>linear</b>: 0, <b>angular</b>: 0}", layout=widgets.Layout(style=font_family_smaller))

status_text = widgets.Label(value="Current Status: Unknown")

past_goals_label = widgets.HTML(value="<b>Past Goals:</b>")
past_goals_display = widgets.HTML()

reached_label = widgets.Label(value="Reached Goals: 0")
unreached_label = widgets.Label(value="Unreached Goals: 100")

In [5]:
# Publisher node function to handle position values and publish them
def publisher_node(msg):
    global pub, old_my_pos_and_vel
    
    # Get the actual position and velocity
    actual_pos = msg.pose.pose.position
    actual_vel_linear = msg.twist.twist.linear
    actual_vel_angular = msg.twist.twist.angular
    
    # Create a Vel message
    my_pos_and_vel = Vel()
    my_pos_and_vel.pos_x = actual_pos.x
    my_pos_and_vel.pos_y = actual_pos.y
    my_pos_and_vel.vel_x = actual_vel_linear.x
    my_pos_and_vel.vel_z = actual_vel_angular.z


    if (abs(my_pos_and_vel.pos_x - old_my_pos_and_vel.pos_x) > 0.05):
        old_my_pos_and_vel.pos_x = my_pos_and_vel.pos_x
    
    
    if (abs(my_pos_and_vel.pos_y -  old_my_pos_and_vel.pos_y) > 0.05):
        old_my_pos_and_vel.pos_y = my_pos_and_vel.pos_y

    if (abs(my_pos_and_vel.vel_x - old_my_pos_and_vel.vel_x) > 0.05):
        old_my_pos_and_vel.vel_x = my_pos_and_vel.vel_x

    if (abs(my_pos_and_vel.vel_z - old_my_pos_and_vel.vel_z) > 0.05):
        old_my_pos_and_vel.vel_z = my_pos_and_vel.vel_z
    
    # Publish the position and velocity
    pub.publish(my_pos_and_vel)

    new_pos_x = "{:.3f}".format(old_my_pos_and_vel.pos_x)
    new_pos_y = "{:.3f}".format(old_my_pos_and_vel.pos_y)
    new_vel_x = "{:.3f}".format(old_my_pos_and_vel.vel_x)
    new_vel_z = "{:.3f}".format(old_my_pos_and_vel.vel_z)    

    
    # Update position and velocity display
    position_display.value = f"<b>POSITION</b>: <b>x</b>: {new_pos_x}, <b>y</b>: {new_pos_y}<br><b>VELOCITY</b>: <b>linear</b>: {new_vel_x}, <b>angular</b>: {new_vel_z}"
    

In [6]:
global old_my_pos_and_vel
old_my_pos_and_vel = Vel()

old_my_pos_and_vel.pos_x = 0.0
old_my_pos_and_vel.pos_y = 0.0
old_my_pos_and_vel.vel_x = 0.0
old_my_pos_and_vel.vel_z = 0.0

# Initialize publisher
pub = rospy.Publisher('/pos_vel', Vel, queue_size=10)

# Initialize subscriber
sub_from_Odom = rospy.Subscriber("/odom", Odometry, publisher_node)

In [7]:
# Function to set goal
def set_goal(input_x, input_y):
    global past_goals, last_goal, goal_reached_flag 
    goal = PlanningGoal()
    rospy.set_param('/des_pos_x', input_x)
    rospy.set_param('/des_pos_y', input_y)
    goal.target_pose.pose.position.x = input_x
    goal.target_pose.pose.position.y = input_y
        
    # Check if the new goal is different from the last goal
    if last_goal is None or (input_x, input_y) != last_goal:
        past_goals.append((input_x, input_y))
        last_goal = (input_x, input_y)
        client.send_goal(goal)
        goal_reached_flag = False
        rospy.loginfo("Inserted goal: des_x = %f, des_y = %f", input_x, input_y)
        update_past_goals_display()  # Update past goals display after setting a new goal
    else:
        rospy.loginfo("New goal is the same as the previous one. Not setting it.")


In [8]:
def cancel_goal():
    global unreached_goals, reached_goals, goal_cancelled
    if client.get_state() == actionlib.GoalStatus.ACTIVE:
        client.cancel_goal()
        rospy.loginfo("Goal cancelled")
        
        unreached_goals += 1  
        
        # PLOT
        sizes = [reached_goals, unreached_goals]
        ax[1].bar(labels, sizes, color=colors, edgecolor='navy')        
        ax[1].set_ylim(0, max(sizes)+3)  # Adjust the y-axis scale                
        
    else:
        rospy.loginfo("No active goal to cancel")

In [9]:
def resume_goal():
    goal = PlanningGoal()
    goal.target_pose.pose.position.x = rospy.get_param('/des_pos_x')
    goal.target_pose.pose.position.y = rospy.get_param('/des_pos_y')
    
    client.send_goal(goal)
    goal_reached_flag = False
    rospy.loginfo("Inserted goal: des_x = %f, des_y = %f", input_x, input_y)
    update_past_goals_display()  # Update past goals display after setting a new goal

In [10]:
# Update status_update function
global goal_reached_flag
    
def status_update(msg):
    global current_status, reached_goals, unreached_goals, ax
    old_value_r = reached_goals
    old_value_u = unreached_goals

        
    if msg.status.status == msg.status.SUCCEEDED: #and not goal_reached_flag:
            rospy.loginfo("reached_goals")
            reached_goals = reached_goals + 1
            goal_reached_flag = True
        
            
    sizes = [reached_goals, unreached_goals]
    labels = 'Reached goals', 'Unreached goals'
    colors = ['lightgreen', 'lightcoral']
    ax[1].bar(labels, sizes, color=colors, edgecolor='navy')  
                       
    ax[1].set_ylim(0, max(sizes)+3)  # Adjust the y-axis scale


In [11]:
client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
client.wait_for_server()

True

In [12]:
# Create widgets for setting goal
global cancel_goal_button

goal_x = widgets.FloatSlider(min=-10.0, max=10.0, step=0.1, description='desired x:', value = 0.0)
goal_y = widgets.FloatSlider(min=-10.0, max=10.0, step=0.1, description='desired y:', value=1.0)
set_goal_button = widgets.Button(description="Set Goal")
cancel_goal_button = widgets.Button(description="Cancel Goal")

In [13]:
# Define actions for button clicks

global cancel_goal_button

def set_goal_click(b):
    set_goal(goal_x.value, goal_y.value)
    cancel_goal_button.description = "Cancel Goal"
    cancel_goal_button.style.button_color = 'lightcoral'
    

def cancel_goal_click(b):
    if(b.description == "Cancel Goal"):
        b.description = "Resume Goal"
        cancel_goal()
        b.style.button_color = 'navajowhite'
        
    else:
        b.description = "Cancel Goal"
        b.style.button_color = 'lightcoral'
        resume_goal()
        
    

In [14]:
# Assign actions to buttons
set_goal_button.on_click(set_goal_click)
cancel_goal_button.on_click(cancel_goal_click)

In [15]:
# Subscribe to goal status updates
goal_status_sub = rospy.Subscriber('/reaching_goal/result', GoalStatus, status_update)

In [16]:
# Update past goals display
def update_past_goals_display():
    global past_goals_display
    goals_str = "<ul>"
    for goal in past_goals:
        goals_str += f"<li>X: {goal[0]}, Y: {goal[1]}</li>"
    goals_str += "</ul>"
    past_goals_display.value = goals_str

In [17]:
# Call update function initially
update_past_goals_display()

In [18]:
# Create widgets for displaying reached and unreached goals
reached_label = widgets.Label(value=f'Reached Goals: {reached_goals}')
unreached_label = widgets.Label(value=f'Unreached Goals: {unreached_goals}')

In [19]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML
from matplotlib.animation import FuncAnimation

global ax, old_my_pos_and_vel

# Font settings
font_family = {'family': 'arial', 'color': 'navy', 'weight': 'bold', 'size': 20}
font_family_smaller = {'family': 'arial', 'color': 'navy', 'weight': 'bold', 'size': 15}

# Define CSS styles for different sections
css_styles = '''
<style>
    .section-title {
        font-family: Arial, sans-serif;
        color: navy;
        font-weight: bold;
        font-size: 30px;
        margin-bottom: 10px;
        margin-left: 200px;
    }
    .section-title-pastgoals {
        font-family: Arial, sans-serif;
        color: navy;
        font-weight: bold;
        font-size: 25px;
        margin-bottom: 10px;
        margin-left: 100px;
        margin-right: 100px;
    }
    .section-title2 {
        font-family: Arial, sans-serif;
        color: navy;
        font-weight: bold;
        font-size: 25px;
        margin-bottom: 10px;
        margin-left: 100px;
        margin-right: 100px;
        
    }
</style>
'''


# Display the section titles using HTML
html_goal_settings = '''
<div class="section-title">Set the GOAL position of the robot</div>
'''
html_control_panel = '''
<div class="section-title2">Control Panel</div>
'''
html_past_goals = '''
<div class="section-title-pastgoals">Past Goals sent by the user</div>
'''
display(HTML(css_styles+html_goal_settings))

# Style widgets 
goal_x.layout.width = '400px'
goal_y.layout.width = '400px'
set_goal_button.style.button_color = 'lightblue'
cancel_goal_button.style.button_color = 'lightcoral'

# status_text.layout.margin = '20px 0'
position_display.layout.margin = '20px 0'
past_goals_display.layout.max_width = '200px'
past_goals_display.layout.max_height = '200px'

# Arrange widgets

goal_setting_widgets = widgets.VBox([widgets.HTML(css_styles+html_control_panel),goal_x, goal_y, widgets.HBox([set_goal_button, cancel_goal_button], layout=widgets.Layout(justify_content='center'))])
  
status_widgets = widgets.VBox([position_display], layout=widgets.Layout(justify_content='center'))

central_column = widgets.VBox([
    goal_setting_widgets,
    status_widgets,
], layout=widgets.Layout(justify_content='center', align_items='center'))

# Arrange widgets for past goals in a VBox
past_goals_section = widgets.VBox([
    widgets.HTML(css_styles+html_past_goals),
    past_goals_display
], layout=widgets.Layout(justify_content='center'))


# display(HTML(css_styles+html_general_title))
# Combine central column and past goals section in an HBox
hbox_layout = widgets.Layout(justify_content='center')
display(widgets.HBox([central_column, past_goals_section], layout=hbox_layout))


%matplotlib widget
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
xdata, ydata = [], []
ln, = ax[0].plot([], [], 'bo', linewidth=1, label='Current position')
scat = ax[0].scatter([], [], color='darkviolet', s=50, label='Ongoing goal')

ax[0].plot([9.5, -9.5, -9.5, 9.5, 9.5], [9.5, 9.5, -9.5, -9.5, 9.5], 'navy', linewidth=5)
ax[0].plot([4, -5, -5], [-0.5, -0.5, -5], 'navy', linewidth=6)
ax[0].plot([-6.5, 5], [4.75, 4.75], 'navy', linewidth=6)


def init():
    ax[0].set_xlim(10, -10)
    ax[0].set_ylim(10, -10)
    ax[0].set_xlabel('x', fontdict=font_family_smaller)
    ax[0].set_ylabel('y', fontdict=font_family_smaller, rotation=0)
    return ln, scat


def update(frame):
    print_pos_x = old_my_pos_and_vel.pos_x
    print_pos_y = old_my_pos_and_vel.pos_y

    xdata.append(print_pos_x)
    ydata.append(print_pos_y)

    ln.set_data(xdata, ydata)
    scat.set_offsets([rospy.get_param('/des_pos_x'), rospy.get_param('/des_pos_y')])
    
    
    return ln, scat


ani = FuncAnimation(fig, update, init_func=init, blit=True)

fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.toolbar_visible = False
fig.canvas.resizable = False

ax[0].set_title('ROBOT LIVE POSITION', fontdict=font_family)
ax[0].grid(True)
ax[0].legend()


# BAR CHART
labels = 'Reached goals', 'Unreached goals'

sizes = [reached_goals, unreached_goals]

colors = ['lightgreen', 'lightcoral']
ax[1].bar(labels, sizes, color=colors, edgecolor='navy')

# Add data labels
#for i, size in enumerate(sizes):
#    ax[1].text(i, size + 1, size, ha='center')

# Add title and adjust layout
ax[1].set_title('GOALS STATISTICS',fontdict=font_family)
ax[1].set_ylim(0, max(sizes)+3)  # Adjust the y-axis scale  # Adjust the y-axis scale
ax[1].set_ylabel("score", fontdict = font_family_smaller)
ax[1].set_xlabel("goals", fontdict = font_family_smaller)

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[INFO] [1713881184.651483, 4688.144000]: Inserted goal: des_x = 5.100000, des_y = 1.000000
[INFO] [1713881213.137926, 4714.235000]: New goal is the same as the previous one. Not setting it.
[INFO] [1713881216.554644, 4717.200000]: Inserted goal: des_x = 6.900000, des_y = 1.000000
[INFO] [1713881219.779094, 4720.052000]: Goal cancelled
